In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [87]:
train_raw = pd.read_csv('/kaggle/input/titanic/train.csv')
features_raw = train_raw.drop(columns=['Survived'])

train_raw.head(5)

import category_encoders as ce


encoder = ce.OneHotEncoder(cols=['Pclass', 'Embarked'])
encoder.fit(features_raw)

print(train_raw.head(5))

def make_nice_features(raw_data):
    raw_data['Embarked'] = raw_data['Embarked'].fillna('S')
    raw_data['Fare'] = raw_data['Fare'].fillna(value=50/raw_data['Pclass'])
    raw_data['Fare'] = raw_data['Fare']/100
    print(raw_data.head(5))
    raw_data['Sex'] = raw_data['Sex'].map({'female': 1, 'male': 0})
    age_median = raw_data['Age'].median()
    raw_data['Age'] = raw_data['Age'].fillna(value=age_median)
    raw_data['Age'] = raw_data['Age']/100
    raw_data['SibSp'] = raw_data['SibSp']/10
    raw_data['Parch'] = raw_data['Parch']/10    
    data = encoder.transform(raw_data)
    data = data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
    return data

cleaned_features = make_nice_features(features_raw)
print(cleaned_features.head(5))

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
  

/opt/conda/lib/python3.10/site-packages/category_encoders/ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
/opt/conda/lib/python3.10/site-packages/category_encoders/ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
/opt/conda/lib/python3.10/site-packages/category_encoders/ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change 

In [88]:
X = cleaned_features
y = train_raw['Survived']

In [89]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(
    X, y, test_size=0.25, random_state=125
)

In [95]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

model = keras.Sequential([
    Dense(20, activation="relu", name="layer1"),
    Dense(1, name="layer2", activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [96]:
history = model.fit(train_x, train_y,
                    batch_size=1,
                    epochs=10,
                    verbose=2,
                    validation_data=(test_x, test_y))

Epoch 1/10
668/668 - 2s - 3ms/step - accuracy: 0.6572 - loss: 0.6083 - val_accuracy: 0.6906 - val_loss: 0.5544
Epoch 2/10
668/668 - 1s - 2ms/step - accuracy: 0.7680 - loss: 0.5040 - val_accuracy: 0.7758 - val_loss: 0.4867
Epoch 3/10
668/668 - 1s - 2ms/step - accuracy: 0.8084 - loss: 0.4659 - val_accuracy: 0.7982 - val_loss: 0.4679
Epoch 4/10
668/668 - 1s - 2ms/step - accuracy: 0.8054 - loss: 0.4529 - val_accuracy: 0.7892 - val_loss: 0.4614
Epoch 5/10
668/668 - 1s - 2ms/step - accuracy: 0.8084 - loss: 0.4444 - val_accuracy: 0.7848 - val_loss: 0.4586
Epoch 6/10
668/668 - 1s - 2ms/step - accuracy: 0.7934 - loss: 0.4475 - val_accuracy: 0.7937 - val_loss: 0.4615
Epoch 7/10
668/668 - 1s - 2ms/step - accuracy: 0.7949 - loss: 0.4417 - val_accuracy: 0.7892 - val_loss: 0.4788
Epoch 8/10
668/668 - 1s - 2ms/step - accuracy: 0.8069 - loss: 0.4439 - val_accuracy: 0.7937 - val_loss: 0.4761
Epoch 9/10
668/668 - 1s - 2ms/step - accuracy: 0.8039 - loss: 0.4443 - val_accuracy: 0.7982 - val_loss: 0.4617
E

Try with competition data

In [ ]:
test_raw = pd.read_csv('/kaggle/input/titanic/test.csv')

test_passenger_id = test_raw['PassengerId']

test_cleaned = make_nice_features(test_raw)

In [ ]:
survived = model.predict(test_cleaned, batch_size=4).round().astype(int)

result = pd.DataFrame({'PassengerId': test_passenger_id, 'Survived': survived.flatten()})


result.to_csv('/kaggle/working/submission.csv', index=False)